In [17]:
import numpy as np
import pandas as pd
from minisom import MiniSom
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# Load the dataset
data = pd.read_csv("labeled_ddostrace.to-victim.20070804.csv")

# Convert other columns to appropriate data types
data['Source IP'] = data['Source IP'].astype(str)
data['Destination IP'] = data['Destination IP'].astype(str)
data['Protocol'] = pd.to_numeric(data['Protocol'], errors='coerce') # if it's numeric
data['Frame Length'] = pd.to_numeric(data['Frame Length'], errors='coerce') # if it's numeric
data['Port Used'] = pd.to_numeric(data['Port Used'], errors='coerce') # if it's numeric
data['Interpacket Time'] = pd.to_numeric(data['Interpacket Time'], errors='coerce') # if it's numeric
data['Entropy'] = pd.to_numeric(data['Entropy'], errors='coerce') # if it's numeric
data['label'] = pd.to_numeric(data['label'], errors='coerce') # if it's numeric

# Drop rows with missing values
data.dropna(inplace=True)

# Separate features and labels
X = data.drop(['label', 'Timestamp', 'Source IP', 'Destination IP'], axis=1).values
y = data['label'].values
print(y)
# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Define SOM parameters
som_grid_rows = 10
som_grid_columns = 10
input_len = X_scaled.shape[1]
num_iterations = 10000

# Train SOM
som = MiniSom(som_grid_rows, som_grid_columns, input_len, sigma=1.0, learning_rate=0.5)
som.random_weights_init(X_scaled)
som.train_batch(X_scaled, num_iterations)

# Find the winning neurons for each sample
winning_neurons = np.array([som.winner(x) for x in X_scaled])

# Assign labels to clusters based on the majority label of the samples in each cluster
cluster_labels = []
total_samples = len(winning_neurons)

for i, neuron in enumerate(winning_neurons):
    cluster_samples_indices = np.where((winning_neurons[:, 0] == neuron[0]) & (winning_neurons[:, 1] == neuron[1]))[0]
    cluster_labels.append(y[cluster_samples_indices].mean())

    # Print progress
    if (i + 1) % 1000 == 0 or i + 1 == total_samples:
        percent_complete = (i + 1) / total_samples * 100
        print(f"Assigning cluster labels: {percent_complete:.2f}% complete")



C:\Users\User\AppData\Local\Temp\ipykernel_20712\4155266547.py:8: DtypeWarning: Columns (0,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("labeled_ddostrace.to-victim.20070804.csv")


[0 1 0 ... 1 0 1]
Assigning cluster labels: 0.09% complete
Assigning cluster labels: 0.18% complete
Assigning cluster labels: 0.26% complete
Assigning cluster labels: 0.35% complete
Assigning cluster labels: 0.44% complete
Assigning cluster labels: 0.53% complete
Assigning cluster labels: 0.62% complete
Assigning cluster labels: 0.71% complete
Assigning cluster labels: 0.79% complete
Assigning cluster labels: 0.88% complete
Assigning cluster labels: 0.97% complete
Assigning cluster labels: 1.06% complete
Assigning cluster labels: 1.15% complete
Assigning cluster labels: 1.23% complete
Assigning cluster labels: 1.32% complete
Assigning cluster labels: 1.41% complete
Assigning cluster labels: 1.50% complete
Assigning cluster labels: 1.59% complete
Assigning cluster labels: 1.67% complete
Assigning cluster labels: 1.76% complete
Assigning cluster labels: 1.85% complete
Assigning cluster labels: 1.94% complete
Assigning cluster labels: 2.03% complete
Assigning cluster labels: 2.12% complet

In [18]:
# Convert labels to binary (assuming 0 for non-DDoS and 1 for DDoS)
y_binary = np.where(y > 0, 1, 0)
# print(y_binary)
# print(cluster_labels)
# Convert cluster labels to binary
cluster_labels_binary = np.where(np.array(cluster_labels) > 0, 1, 0)

In [19]:
# Evaluate accuracy
accuracy = accuracy_score(y_binary, cluster_labels_binary)
print("Accuracy:", accuracy)

Accuracy: 0.8621953412284011


In [3]:
# # Evaluate accuracy
# accuracy = accuracy_score(y, cluster_labels)
# print("Accuracy:", accuracy)


ValueError: Classification metrics can't handle a mix of binary and continuous targets